# Интегрирование

Пакет ```scipy.integrate``` содержит методы для вычисления определенных интегралов. С их помощью можно вычислять собственные (с конечными пределами) и несобственные (с бесконечными пределами) интегралы. Эти методы также позволяют выполнять численное интегрирование систем обыкновенных дифференциальных уравнений.

## Определенные интегралы от одной переменной

Основной программой численного интегрирования является ```scipy.integrate.quad```, основанная на старой «заслуженной» библиотеке FORTRAN 77 QUADPACK. Здесь используется адаптивная квадратура для приближенного вычисления значения интеграла посредством деления его области интегрирования на меньшие интервалы, выбираемые итеративно для соответствия
конкретному пределу допускаемой погрешности (т. е. оценочной абсолютной или относительной погрешности). В  самой простой форме метод принимает три аргумента: объект функции Python, соответствующий интегрируемой функции, func и пределы интегрирования ```a``` и ```b```. В аргументе ```func``` обязательно должен передаваться хотя бы один объект, если этот аргумент содержит более одного объекта, то интегрирование выполняется по координате, соответствующей первому значению аргумента. При простом способе применения удобным способом определения func являются lambda-выражения. Например, для получения значения интеграла 

![image1](images/image1.png)


в численном виде:

In [7]:
from scipy.integrate import quad

In [8]:
 f = lambda x: 1/x**2

In [10]:
 quad(f, 1, 4)

(0.7500000000000002, 1.913234548258995e-09)

метод ```quad``` возвращает два значения в кортеже – значение интеграла и оценку
абсолютной погрешности полученного результата.

Для вычисления несобственных интегралов используется специальное значение ```np.inf```:

In [12]:
import numpy as np
quad(lambda x: np.exp(-x**2), 0, np.inf)

(0.8862269254527579, 7.101318390915439e-09)

In [13]:
 np.sqrt(np.pi)/2 # Результат, полученный аналитическим способом.

0.8862269254527579

Обратите внимание: при этом вызове ```quad``` даже не  указывается имя интегрируемой функции, а  просто передается сама функция как анонимный ```lambda```-объект.
Для более сложных функций требуется явное определение объекта функции Python с помощью ключевого слова ```def```:

In [15]:
def g(x):
    if abs(x) < 0.5:
        return -x
    return x - np.sign(x)

In [16]:
quad(g, -0.6, 0.8)

(-0.060000000000000026, 6.661338147750943e-17)

Функции с сингулярными (особыми, критическими) точками или с точками разрыва могут создавать проблемы для программы численного интегрирования, даже если требуемый интеграл однозначно определен. Например, функция ```sinc``` (кардинальный синус) ```f(x) = sin(x)/x``` имеет устранимую особую точку при ```x = 0```, в которой при обычном применении метода quad возникает критическая ошибка:

In [17]:
sinc = lambda x: np.sin(x)/x

In [18]:
quad(sinc, -2, 2)

C:\Users\user\AppData\Local\Temp\ipykernel_13108\3926713972.py:1: RuntimeWarning: invalid value encountered in double_scalars
  sinc = lambda x: np.sin(x)/x
C:\Users\user\AppData\Local\Temp\ipykernel_13108\4037327689.py:1: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad(sinc, -2, 2)


(nan, nan)

Устранить эту проблему можно, передавая в метод quad список таких точек
разрыва в аргументе ```points``` (список не обязательно должен быть упорядоченным):

In [20]:
quad(sinc, -2, 2, points=[0,])

(3.210825953605389, 3.5647329017567276e-14)

Следует отметить, что точки разрыва не могут быть заданы при бесконечных пределах интегрирования.

Аргументы ```epsrel``` и ```epsabs``` позволяют определить требуемую точность интегрирования как относительную и  абсолютную погрешность соответственно. По умолчанию для обоих аргументов принято значение ```1.49e-8```, но интегрирование можно выполнить быстрее, если нужно получить менее точный результат.

В качестве примера рассмотрим быстро изменяющуюся функцию ```f(x) = e −|x|sin^2 x^2```:

In [22]:
f = lambda x: np.sin(x**2)**2 * np.exp(-np.abs(x))

In [23]:
quad(f, -1, 2, epsabs=0.1)

(0.29551455828969975, 0.0015295718279094228)

In [24]:
quad(f, -1, 2, epsabs=1.49e-8) # (Абсолютная погрешность, принятая по умолчанию.)

(0.29551455505239044, 4.4497633151515266e-10)

Обратите внимание: значение ```epsabs``` – это только требуемая верхняя граница: действительная оцениваемая точность результата может быть гораздо лучшей, т. е. в действительности полученный результат может оказаться более точным, чем эта оценка.
Если функция принимает один или несколько параметров в  дополнение к своему основному аргументу, то эти дополнительные параметры необходимо передать в метод quad как кортеж в аргументе ```args```. Например, интеграл

![image2](images/image2.png)

In [26]:
def f(x, n, m):
    return np.sin(x)**n * np.cos(x)**m

In [27]:
n, m = 2, 1

In [28]:
quad(f, -np.pi/2, np.pi/2, args=(n, m))

(0.6666666666666667, 1.6257070626918945e-13)

Обратите внимание: здесь дополнительные параметры ```n```  и ```m```  передаются
как аргументы интегрируемой функции после координаты, определяющей направление интегрирования по (x).

**Пример 1**. Рассмотрим тор со средним радиусом ```R``` и радиусом поперечного
сечения ```r```. Объем этой фигуры можно вычислить аналитически в декартовых координатах как объем тела вращения:

![image3](images/image3.png)

Центр тора является началом координат, а ось ```z``` принимается за ось симметрии.
Вычисление интеграла утомительно, но приводит к стандартным методам:
```V = 2π^2Rr^2```. Здесь мы применим численный метод с использованием значений
```R = 4```, ```r = 1```:

In [29]:
R, r = 4, 1

In [30]:
f = lambda x, R, r: x * np.sqrt(r**2 - (x-R)**2)

In [31]:
V, _ = quad(f, R-r, R+r, args=(R, r))

In [32]:
V *= 4 * np.pi

In [33]:
Vexact = 2 * np.pi**2 * R * r**2

In [34]:
print('V = {} (exact: {})'.format(V, Vexact))

V = 78.95683520871498 (exact: 78.95683520871486)


## Интегралы от двух и нескольких переменных

В модуле ```scipy.integrate``` методы ```dblquad```, ```tplquad``` и ```nquad``` вычисляют двойные, тройные и кратные интегралы соответственно. Поскольку в общем случае
пределы по одной координате могут зависеть от другой координаты, синтаксис вызова перечисленных выше методов немного сложнее.

Метод ```dblquad``` вычисляет двойной интеграл:

![image4](images/image4.png)

Здесь ```f(x,y)``` передается как функция по меньшей мере двух переменных ```func(y, x, …)```. Эта фуункция обязательно должна принимать ```y``` как свой первый аргумент, а  ```x``` как второй аргумент. Пределы интегрирования передаются в  ```dblquad``` в  следующих четырех аргументах. Сначала два аргумента ```a```  и  ```b``` определяют нижний и верхний пределы интегрирования по ```x```, как и для метода ```quad```. Следующие два аргумента ```gfun``` и  ```hfun``` определяют нижний и  верхний пределы интегрирования по ```y```, и они обязательно должны быть вызываемыми объектами, принимающими один аргумент  – число с  плавающей точкой, значение x, при котором этот предел применяется (т.е. эти аргументы сами обязательно должны быть функциями от ```x```). Если любой из пределов
интегрирования по ```y``` не зависит от ```x```, то ```gfun``` или ```hfun``` могут возвращать постоянное значение.

Как простой пример рассмотрим интеграл

![image5](images/image5.png)






In [36]:
f = lambda y, x: x**2 * y

In [38]:
a, b = 1, 4

In [39]:
gfun = lambda x: 0

In [40]:
hfun = lambda x: 2

In [42]:
from scipy.integrate import dblquad
dblquad(f, a, b, gfun, hfun)

(42.00000000000001, 4.662936703425658e-13)

Здесь оба аргумента ```gfun``` и ```hfun``` вызываются с передачей в них значения ```x```,
но они возвращают постоянные значения (0 и 2 соответственно) вне зависимости от переданного значения.

Разумеется, все показанные выше операции в исходном коде можно свернуть в одну строку:

In [43]:
dblquad(lambda y, x: x**2 * y, 1, 4, lambda x: 0, lambda x: 2)

(42.00000000000001, 4.662936703425658e-13)

Двойной интеграл можно использовать для вычисления площади некоторой двумерной фигуры, ограниченной одной или несколькими функциями.

Для примера в  полярных координатах рассмотрим область внутри кривой
```r  =  2  +  2sinθ```, но вне окружности с радиусом ```r  = 2``` при ```θ```  в интервале ```[0, 2π]``` (см. рис. 1).

![image6](images/image6.png)

**Рис. 1**. Фигура, определенная как область внутри кривой ```r=2 + 2sinθ```, но вне окружности с радиусом ```r=2```

Эти фигуры пересекаются в точках ```θ  = 0, π```, поэтому требуемый интеграл
имеет следующий вид:

![image7](images/image7.png)

где ```r dr dθ``` – бесконечно малый элемент области в полярных координатах. Этот
частный определенный интеграл без затруднений вычисляется аналитически ```(A = 8 + π)```, поэтому результат применения численного метода легко проверить:

In [46]:
r1, r2 = lambda theta: 2, lambda theta: 2 + 2*np.sin(theta)

In [47]:
A, _ = dblquad(lambda r, theta: r, 0, np.pi, r1, r2)

In [48]:
8 + np.pi # Точный результат.

11.141592653589793

Вычисляемая функция – это просто ```r```, определяемая выражением ```lambda r```,
```theta: r```. Во внутреннем интеграле пределы по ```r``` равны 2 и 2 + 2sinθ. Для внешнего интеграла значение угла ```θ``` определяется от 0 до π.
Метод ```tplquad``` вычисляет тройные интегралы и принимает функцию от трех переменных ```func(z, y, x)``` и шесть дополнительных аргументов: постоянные пределы интегрирования по ```x``` ```a``` и  ```b```, пределы интегрирования по ```y``` ```gfun(x)```
и ```hfun(x)``` (которые являются функциями, как и для метода ```dblquad```) и пределы
интегрирования по ```z``` ```qfun(x, y)``` и ```rfun(x, y)``` (функции от ```x``` и ```y``` именно в таком порядке).
Интегрирование с более высокой кратностью выполняется методом ```scipy.integrate.nquad```, который здесь не рассматривается (документацию и примеры см. здесь: https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.
nquad.html).

**Пример  2**. Объем единичной сферы ```4π/3``` можно записать в  виде тройного
интеграла в сферических полярных координатах с постоянными пределами интегрирования:


![image8](images/image8.png)

In [49]:
from scipy.integrate import tplquad

In [50]:
tplquad(lambda phi, theta, r: r**2 * np.sin(theta),
 0, 1,
 lambda theta: 0, lambda theta: np.pi,
 lambda theta, phi: 0, lambda theta, phi: 2*np.pi)

(4.18879020478639, 1.389095079694993e-13)

Кроме того, можно записать это выражение в  декартовых координатах с пределами интегрирования в форме функций:

![image9](images/image9.png)

где интеграл находится в положительном октанте трехмерной декартовой системы координат.

In [51]:
A, _ = tplquad(lambda z, y, x: 1,
 0, 1,
 lambda x: 0, lambda x: np.sqrt(1 - x**2),
 lambda x, vy: 0, lambda x, y: np.sqrt(1 - x**2 - y**2))

In [52]:
8*A

4.1887902047939845

**Пример 3**. В этом примере определяется масса и центр масс тетраэдра, ограниченного координатными осевыми плоскостями и плоскостью ```x + y + z = 1``` с плотностью ```ρ = ρ(x,y,z)```, где плотность ```ρ(x,y,z)``` представлена как лямбда-функция. Проверим
это на функциях ```ρ = 1```, ```ρ = x``` и ```ρ = x2 + y2+ z2```.

Масса может быть записана в виде тройного интеграла плотности в зависимости от объема тетраэдра:

![image10](images/image10.png)

а координаты центра масс определяются следующими интегралами:

![image11](images/image11.png)

Программа в  листинге 1 использует метод ```scipy.integrate.tplquad``` для
выполнения требуемых операций интегрирования (которые могут быть решены и аналитически).

**Листинг 1**. Вычисление массы и центра масс тетраэдра с учетом его трех различных плотностей

In [54]:
# eg8-tetrahedron-cofm.py
import numpy as np
from scipy.integrate import tplquad
# Пределы интегрирования по x, y, z.
a, b = 0, 1
gfun, hfun = lambda x: 0, lambda x: 1 - x
qfun, rfun = lambda x, y: 0, lambda x, y: 1 - x - y
lims = (a, b, gfun, hfun, qfun, rfun)
# Три различные функции плотности.
rhos = [lambda x, y, z: 1, lambda x, y, z: x, lambda x, y, z: x**2 + y**2 + z**2]
for rho in rhos:
    # Масса как тройной интеграл от rho по объему.
    m, _ = tplquad(rho, *lims)
    # Центр масс (xbar, ybar, zbar).
    mxbar, _ = tplquad(lambda x, y, z: x * rho(x, y, z), *lims)
    mybar, _ = tplquad(lambda x, y, z: y * rho(x, y, z), *lims)
    mzbar, _ = tplquad(lambda x, y, z: z * rho(x, y, z), *lims)
    xbar, ybar, zbar = mxbar / m, mybar / m, mzbar / m
    print('mass = {:g}, CofM = ({:g}, {:g}, {:g})'.format(m, xbar, ybar, zbar))

mass = 0.166667, CofM = (0.25, 0.25, 0.25)
mass = 0.0416667, CofM = (0.4, 0.2, 0.2)
mass = 0.05, CofM = (0.277778, 0.277778, 0.277778)


Обратите внимание: шесть аргументов, представляющих пределы тройного интеграла (две константы и две пары лямбда-функций), упакованы в кортеж ```lims``` (здесь скобки не обязательны).


In [1]:
import numpy as np
from scipy.integrate import quad

## Упражнения

1. Использовать метод ```scipy.integrate.quad``` для вычисления следующего интеграла:

![image12](images/image12.png)

2 Использовать метод ```scipy.integrate.quad``` для вычисления следующих
определенных интегралов (большинство из них можно также представить
в конечной форме в заданном интервале, но это затруднительно).

а) ![image13](images/image13.png)

In [7]:
f1 = lambda x: x**4 * (1 -x)**4/(1 + x**2)
quad(f1, 0, 1)

(0.0012644892673496185, 1.1126990906558069e-14)

In [8]:
22/7 - np.pi

0.0012644892673496777

б) Этот интеграл применяется в теории (модели) Дебая, описывающей теплоемкость кристаллических веществ при низких температурах:

![image14](images/image14.png)

In [10]:
f2 = lambda x: x**3/(np.exp(x) - 1)
quad(f2, 0, np.inf)

<ipython-input-10-302d287139ce>:1: RuntimeWarning: overflow encountered in exp
  f2 = lambda x: x**3/(np.exp(x) - 1)


(6.49393940226683, 2.62847130244751e-09)

In [11]:
np.pi**4 / 15

6.493939402266828

в) Интеграл, который иногда называют мечтой второкурсника:

![image15](images/image15.png)

In [12]:
f3 = lambda x: x**-x
quad(f3, 0, 1)

(1.2912859970626636, 3.668487735808412e-11)

In [13]:
n, I, TOL = 0, 0, 1.e-16
while True:
    Iold = I
    n += 1
    I += n**-n
    if I-Iold < TOL:
        break

I

1.2912859970626636

г) ![image16](images/image16.png)

In [24]:
from scipy.misc import factorial
f4 = lambda x, p: np.log(1/x)**p

for p in range(10):
    print(quad(f4, 0, 1, args=(p,))[0], factorial(p))

ImportError: cannot import name 'factorial' from 'scipy.misc' (C:\ProgramData\Anaconda3\lib\site-packages\scipy\misc\__init__.py)

д) ![image17](images/image17.png)

In [17]:
from scipy.special import i0
z = np.linspace(0, 2, 100)
y1 = i0(z)
f5 = lambda theta, z: np.exp(z*np.cos(theta))
y2 = np.array([quad(f5, 0, 2*np.pi, args=(zz,))[0] for zz in z])
y2 /= 2 * np.pi
np.max(abs(y2-y1))

3.4796610037801656e-12